In [1]:
import pandas as pd
import numpy as np
from scipy import io
import os, errno

In [2]:
# consolidate the true trials tDetect and exploration time across ALL the data.

direc1 = r'E:\Downloads\ManducaMultiSenseData\Step3'
files = [file for file in os.listdir(direc1) if file.endswith('.csv')]
print(files)


# load all the files with IRdtect data #L0.1_c-3_m45, L0.1_c-3_m46, L50_c-3_m50 all have their IR data stitched together so either manually stitch back together with code at bottom of script or use stitched step 4 files on lab computre
direc2 = r'E:\Downloads\ManducaMultiSenseData\Step4'
IR_data_files = [file for file in os.listdir(direc2) if file.endswith('.npy')]
IR_data_files

['L0.1_c-3_m10_MothInOut_labeled.csv', 'L0.1_c-3_m12_MothInOut_labeled.csv', 'L0.1_c-3_m20_MothInOut_labeled.csv', 'L0.1_c-3_m21_MothInOut_labeled.csv', 'L0.1_c-3_m22_MothInOut_labeled.csv', 'L0.1_c-3_m23_MothInOut_labeled.csv', 'L0.1_c-3_m24_MothInOut_labeled.csv', 'L0.1_c-3_m25_MothInOut_labeled.csv', 'L0.1_c-3_m27_MothInOut_labeled.csv', 'L0.1_c-3_m2_MothInOut_labeled.csv', 'L0.1_c-3_m32_MothInOut_labeled.csv', 'L0.1_c-3_m34_MothInOut_labeled.csv', 'L0.1_c-3_m37_MothInOut_labeled.csv', 'L0.1_c-3_m38_MothInOut_labeled.csv', 'L0.1_c-3_m39_MothInOut_labeled.csv', 'L0.1_c-3_m40_MothInOut_labeled.csv', 'L0.1_c-3_m41_MothInOut_labeled.csv', 'L0.1_c-3_m43_MothInOut_labeled.csv', 'L0.1_c-3_m44_MothInOut_labeled.csv', 'L0.1_c-3_m45_MothInOut_labeled.csv', 'L0.1_c-3_m46_MothInOut_labeled.csv', 'L0.1_c-3_m47_MothInOut_labeled.csv', 'L0.1_c-3_m48_MothInOut_labeled.csv', 'L0.1_c-3_m49_MothInOut_labeled.csv', 'L0.1_c-3_m50_MothInOut_labeled.csv', 'L0.1_c-3_m54_MothInOut_labeled.csv', 'L0.1_c-3_m5

['L0.1_c-3_m10.mat_IRdetect.npy',
 'L0.1_c-3_m12.mat_IRdetect.npy',
 'L0.1_c-3_m2.mat_IRdetect.npy',
 'L0.1_c-3_m20.mat_IRdetect.npy',
 'L0.1_c-3_m21.mat_IRdetect.npy',
 'L0.1_c-3_m22.mat_IRdetect.npy',
 'L0.1_c-3_m23.mat_IRdetect.npy',
 'L0.1_c-3_m24.mat_IRdetect.npy',
 'L0.1_c-3_m25.mat_IRdetect.npy',
 'L0.1_c-3_m27.mat_IRdetect.npy',
 'L0.1_c-3_m32.mat_IRdetect.npy',
 'L0.1_c-3_m34.mat_IRdetect.npy',
 'L0.1_c-3_m37.mat_IRdetect.npy',
 'L0.1_c-3_m38.mat_IRdetect.npy',
 'L0.1_c-3_m39.mat_IRdetect.npy',
 'L0.1_c-3_m40.mat_IRdetect.npy',
 'L0.1_c-3_m41.mat_IRdetect.npy',
 'L0.1_c-3_m43.mat_IRdetect.npy',
 'L0.1_c-3_m44.mat_IRdetect.npy',
 'L0.1_c-3_m45.mat_IRdetect.npy',
 'L0.1_c-3_m46.mat_IRdetect.npy',
 'L0.1_c-3_m47.mat_IRdetect.npy',
 'L0.1_c-3_m48.mat_IRdetect.npy',
 'L0.1_c-3_m49.mat_IRdetect.npy',
 'L0.1_c-3_m5.mat_IRdetect.npy',
 'L0.1_c-3_m50.mat_IRdetect.npy',
 'L0.1_c-3_m54.mat_IRdetect.npy',
 'L0.1_c-3_m57.mat_IRdetect.npy',
 'L0.1_c-3_m8.mat_IRdetect.npy',
 'L50_c-3_m10.mat

In [4]:
new_path = r'E:\Downloads\ManducaMultiSenseData\Step5'

try:
    os.makedirs(new_path)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

for fname in files:
    df = pd.read_csv(direc1 + '/' + fname)
    # pull out all the trueTrials and reset the index to default to get true trial #
    trueTrials = df[df.label == 1].loc[:, ['In_Frame', 'Out_Frame']].reset_index(drop = True)
    
    
    # load IR files and get indices of all the frames where IRDetect == 1
    
    IR_detect_file = [file for file in IR_data_files if fname[:-22] in file]

    print(IR_detect_file[0])
    loadedIRdetect = np.load(direc2 + '/' + IR_detect_file[0])
    indx = np.where(loadedIRdetect)
    indx = indx[0].reshape(-1)

    # for each true trial, find the first frame where IRDetect == True
    proboscisDetect = []
    discoveryTime = []
    for framein, frameout in zip(trueTrials.In_Frame, trueTrials.Out_Frame):
        allframes = [frame for frame in indx if framein < frame < frameout]
        if allframes == []:
            detect = np.nan
        else:
            detect = min(allframes)
        Time  = detect - framein
        proboscisDetect.append(detect)
        discoveryTime.append(Time)

    df1 = pd.DataFrame({'ProboscisDetect': proboscisDetect})
    df2 = pd.DataFrame({'DiscoveryTime' : discoveryTime})

    new_data = pd.concat([trueTrials, df1, df2], axis = 1)
    new_data.to_csv(new_path + "\\" + fname[:-22] + '_RawDataForExplorationTime.csv')

L0.1_c-3_m10.mat_IRdetect.npy
L0.1_c-3_m12.mat_IRdetect.npy
L0.1_c-3_m20.mat_IRdetect.npy
L0.1_c-3_m21.mat_IRdetect.npy
L0.1_c-3_m22.mat_IRdetect.npy
L0.1_c-3_m23.mat_IRdetect.npy
L0.1_c-3_m24.mat_IRdetect.npy
L0.1_c-3_m25.mat_IRdetect.npy
L0.1_c-3_m27.mat_IRdetect.npy
L0.1_c-3_m2.mat_IRdetect.npy
L0.1_c-3_m32.mat_IRdetect.npy
L0.1_c-3_m34.mat_IRdetect.npy
L0.1_c-3_m37.mat_IRdetect.npy
L0.1_c-3_m38.mat_IRdetect.npy
L0.1_c-3_m39.mat_IRdetect.npy
L0.1_c-3_m40.mat_IRdetect.npy
L0.1_c-3_m41.mat_IRdetect.npy
L0.1_c-3_m43.mat_IRdetect.npy
L0.1_c-3_m44.mat_IRdetect.npy
L0.1_c-3_m45.mat_IRdetect.npy
L0.1_c-3_m46.mat_IRdetect.npy
L0.1_c-3_m47.mat_IRdetect.npy
L0.1_c-3_m48.mat_IRdetect.npy
L0.1_c-3_m49.mat_IRdetect.npy
L0.1_c-3_m50.mat_IRdetect.npy
L0.1_c-3_m54.mat_IRdetect.npy
L0.1_c-3_m57.mat_IRdetect.npy
L0.1_c-3_m5.mat_IRdetect.npy
L0.1_c-3_m8.mat_IRdetect.npy
L50_c-3_m10.mat_IRdetect.npy
L50_c-3_m12.mat_IRdetect.npy
L50_c-3_m13.mat_IRdetect.npy
L50_c-3_m14.mat_IRdetect.npy
L50_c-3_m15.mat_I